In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

examples = [
    {
        "movie": "Top Gun",
        "answer": """
        🛩️👨‍✈️🔥
        """,
    },
    {
        "movie": "The Godfather",
        "answer": """
        👨‍👨‍👦🔫🍝
        """,
    },
    {
        "movie": "The Martian",
        "answer": """
        🚀👨‍🚀🌌
        """,
    },
]

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What emoji can describe this {movie}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert, you give an answer with 3 emojis."),
        example_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "What emoji can you describe this {movie} with?"),
    ]
)

def load_memory(_):
  return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(movie):
    result = chain.invoke({"movie": movie})
    memory.save_context(
        {"input": movie},
        {"output": result.content},
    )
    print(result)
    
invoke_chain("Interstellar")
invoke_chain("Gravity")

🚀🌌⏳content='🚀🌌⏳'
🌌👩‍🚀🌠content='🌌👩\u200d🚀🌠'


In [2]:
invoke_chain("What was the first movie I asked?")

🛩️👨‍✈️🔥content='🛩️👨\u200d✈️🔥'


In [3]:
invoke_chain("What was the second movie I asked?")

👨‍👨‍👦🔫🍝content='👨\u200d👨\u200d👦🔫🍝'


In [4]:
invoke_chain("What was the fourth movie I asked?")

🚀👨‍🚀🌌The human mentions "Interstellar" and the AI responds with rocket, galaxy, and time emojis. The human then brings up "Gravity" and the AI responds with galaxy, astronaut, and star emojis.content='🚀👨\u200d🚀🌌'
